In [ ]:
import cv2
import numpy as np
import imutils
import easyocr
import os
import pathlib
import tensorflow as tf

from matplotlib import pyplot as plt


In [ ]:
def character_recognition(img, location): 
    if len(location) > 0:
        tmp=img[location[0]: location[1], location[2]: location[3] ] #crop the image by given coordinates
        image=cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)

        reader = easyocr.Reader(['en'], gpu=False)
        result = reader.readtext(image)
        print(result)

In [ ]:
def delete(file):
    os.remove('/Users/milovan/Desktop/PlateRecognitionV2' + '/' + file)

In [ ]:
# PROVIDE PATH TO VIDEO DIRECTORY
VIDEO_PATHS = '/Users/milovan/Desktop/PlateRecognitionV2/videoTrain.avi'

# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = '/Users/milovan/Desktop/PlateRecognitionV2/my_model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = '/Users/milovan/Desktop/PlateRecognitionV2/label-map.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = 0.50

# Load the model
# ~~~~~~~~~~~~~~
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# Load label map data (for plotting)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,use_display_name=True)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    """
    return np.array(Image.open(path))

print('Running inference for {}... '.format(VIDEO_PATHS), end='')

video = cv2.VideoCapture(VIDEO_PATHS)
while(video.isOpened()):

    # Acquire frame and expand frame dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    ret, frame = video.read()
    
    cv2.imwrite("tempFrame.jpg", frame)
    
    image = cv2.imread("tempFrame.jpg")
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    frame_expanded = np.expand_dims(frame_rgb, axis=0)
    
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(frame)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
            
    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    frame_with_detections = frame.copy()
    
    # SET MIN SCORE THRESH TO MINIMUM THRESHOLD FOR DETECTIONS
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
          frame_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=MIN_CONF_THRESH,
          agnostic_mode=False)
    
    boxes = detections['detection_boxes'] 
    scores = detections['detection_scores']
    min_score_tresh = 0.5
    bboxes = boxes[scores > min_score_tresh]
    width, height = image.shape[:2]
    final_box=[]
    for box in bboxes:
        ymin, xmin, ymax, xmax = box
        final_box = [int(ymin * width), int(xmin * height), int(ymax * width), int(xmax *height)]
    
    location = final_box
    
    cv2.imshow('Object Detector', frame_with_detections)
    
    character_recognition(image, location)
    
    delete('tempFrame.jpg')

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()
print("Done")